# Base Model✅

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class DataHandler:
    def __init__(self):
        self.scaler = MinMaxScaler()
    
    def fetch_data(self, symbol, start_date, end_date):
        print(f"Fetching data for {symbol}...")
        df = yf.download(symbol, start=start_date, end=end_date)
        if df.empty:
            raise ValueError(f"No data found for {symbol}")
        return self.prepare_data(df)
    
    def prepare_data(self, df):
        try:
            # Create copy to avoid SettingWithCopyWarning
            df = df.copy()
            
            # Convert to 1D numpy arrays
            close_prices = df['Close'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            volume = df['Volume'].values.astype(float).flatten()
            
            # Technical Indicators
            df['Returns'] = df['Close'].pct_change()
            df['SMA_20'] = talib.SMA(close_prices, timeperiod=20)
            df['EMA_20'] = talib.EMA(close_prices, timeperiod=20)
            df['RSI'] = talib.RSI(close_prices, timeperiod=14)
            
            # MACD
            macd, signal, _ = talib.MACD(close_prices)
            df['MACD'] = macd
            df['MACD_signal'] = signal
            
            # Bollinger Bands
            bb_upper, bb_middle, bb_lower = talib.BBANDS(close_prices)
            df['BB_upper'] = bb_upper
            df['BB_middle'] = bb_middle
            df['BB_lower'] = bb_lower
            
            # Volume indicators
            df['OBV'] = talib.OBV(close_prices, volume)
            
            # Momentum
            df['MOM'] = talib.MOM(close_prices, timeperiod=14)
            
            # Handle NaN values and normalize
            features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                       'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 
                       'BB_lower', 'OBV', 'MOM']
            
            df[features] = df[features].fillna(method='ffill').fillna(method='bfill')
            df[features] = self.scaler.fit_transform(df[features])
            
            print(f"Data shape after preparation: {df.shape}")
            return df.dropna()
            
        except Exception as e:
            print(f"Error in prepare_data: {str(e)}")
            raise

class TradingEnvironment:
    def __init__(self, data, initial_balance=100000):
        if data is None or data.empty:
            raise ValueError("Data cannot be empty")
            
        self.data = data
        self.initial_balance = initial_balance
        self.features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI',
                        'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                        'BB_lower', 'OBV', 'MOM']
        
        if not all(f in self.data.columns for f in self.features):
            missing = [f for f in self.features if f not in self.data.columns]
            raise ValueError(f"Missing features in data: {missing}")
            
        self.reset()
    
    def reset(self):
        self.balance = self.initial_balance
        self.position = 0
        self.current_step = 0
        self.portfolio_value_history = [self.initial_balance]
        return self._get_state()
    
    def _get_state(self):
        return self.data.iloc[self.current_step][self.features].values
    
    def step(self, action):
        # Get current price as float
        current_price = float(self.data.iloc[self.current_step]['Close'])
        
        # Store previous portfolio value for reward calculation
        prev_portfolio_value = self.portfolio_value_history[-1]
        
        # Execute trade with zero price protection
        if current_price > 0:  # Only execute trades if price is valid
            if action == 0 and self.position > 0:  # Sell
                self.balance += current_price * self.position
                self.position = 0
            elif action == 2 and self.position == 0:  # Buy
                shares = self.balance / current_price
                self.position = shares
                self.balance = 0
        
        # Move to next step
        self.current_step += 1
        
        # Calculate new portfolio value
        portfolio_value = self.balance + (self.position * current_price)
        self.portfolio_value_history.append(portfolio_value)
        
        # Calculate reward with safe division
        if prev_portfolio_value > 0:
            reward = (portfolio_value - prev_portfolio_value) / prev_portfolio_value
        else:
            reward = 0
        
        # Check if done
        done = self.current_step >= len(self.data) - 1
        
        return self._get_state(), reward, done

class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        
        # Neural network layers
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)
        
        self.optimizer = optim.Adam(self.parameters())
        self.criterion = nn.MSELoss()
        self.to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(device)
            act_values = self(state)
            return torch.argmax(act_values).item()
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return 0
        
        minibatch = random.sample(self.memory, self.batch_size)
        
        states = torch.FloatTensor([i[0] for i in minibatch]).to(device)
        actions = torch.LongTensor([i[1] for i in minibatch]).to(device)
        rewards = torch.FloatTensor([i[2] for i in minibatch]).to(device)
        next_states = torch.FloatTensor([i[3] for i in minibatch]).to(device)
        dones = torch.FloatTensor([i[4] for i in minibatch]).to(device)
        
        # Current Q values
        current_q_values = self(states).gather(1, actions.unsqueeze(1))
        
        # Next Q values
        with torch.no_grad():
            next_q_values = self(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values
        
        # Compute loss and update
        loss = self.criterion(current_q_values.squeeze(), target_q_values)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # Update epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
        return loss.item()

def test_agent(agent, test_data, initial_balance=10000):
    class TestEnvironment(TradingEnvironment):
        def __init__(self, data, initial_balance):
            super().__init__(data, initial_balance)
    
    # Create test environment
    test_env = TestEnvironment(test_data, initial_balance)
    
    # Disable exploration during testing
    original_epsilon = agent.epsilon
    agent.epsilon = 0
    
    # Run test episode
    state = test_env.reset()
    done = False
    actions_taken = []
    
    while not done:
        with torch.no_grad():
            action = agent.act(state)
            actions_taken.append(action)
            state, reward, done = test_env.step(action)
    
    # Restore original epsilon
    agent.epsilon = original_epsilon
    
    return test_env.portfolio_value_history, actions_taken

def plot_results(train_values, test_values, training_losses):
    fig = go.Figure()

    # Plot portfolio values
    fig.add_trace(go.Scatter(
        x=list(range(len(train_values))),
        y=train_values,
        mode='lines',
        name='Training Portfolio'
    ))

    fig.add_trace(go.Scatter(
        x=list(range(len(train_values), len(train_values) + len(test_values))),
        y=test_values,
        mode='lines',
        name='Testing Portfolio'
    ))

    fig.update_layout(
        title='Portfolio Value Over Time',
        xaxis_title='Trading Days',
        yaxis_title='Portfolio Value ($)',
        legend_title='Legend'
    )

    # Plot training loss
    fig.add_trace(go.Scatter(
        x=list(range(len(training_losses))),
        y=training_losses,
        mode='lines',
        name='Training Loss',
        yaxis='y2'
    ))

    fig.update_layout(
        yaxis2=dict(
            title='Loss',
            overlaying='y',
            side='right'
        ),
        title='Training Loss Over Episodes',
        xaxis_title='Episode',
        yaxis_title='Loss',
        legend_title='Legend'
    )

    fig.show()

def train_agent(env, agent, episodes):
    training_losses = []
    best_reward = float('-inf')
    
    for episode in tqdm(range(episodes), desc="Training"):
        state = env.reset()
        total_reward = 0
        episode_losses = []
        
        while True:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            
            agent.remember(state, action, reward, next_state, done)
            loss = agent.replay()
            
            if loss > 0:
                episode_losses.append(loss)
            
            state = next_state
            total_reward += reward
            
            if done:
                avg_loss = np.mean(episode_losses) if episode_losses else 0
                training_losses.append(avg_loss)
                
                if total_reward > best_reward:
                    best_reward = total_reward
                    torch.save(agent.state_dict(), 'best_model.pth')
                
                print(f"\nEpisode: {episode + 1}/{episodes}")
                print(f"Total Reward: {total_reward:.2f}")
                print(f"Average Loss: {avg_loss:.4f}")
                print(f"Epsilon: {agent.epsilon:.4f}")
                print(f"Final Portfolio Value: ${env.portfolio_value_history[-1]:.2f}")
                break
    
    return training_losses

if __name__ == "__main__":
    try:
        # Set random seeds for reproducibility
        torch.manual_seed(42)
        np.random.seed(42)
        random.seed(42)
        
        # Parameters
        SYMBOL = 'AAPL'
        START_DATE = '2020-01-01'
        END_DATE = '2024-02-11'
        EPISODES = 100
        
        print("Initializing trading simulation...")
        
        # Initialize components
        data_handler = DataHandler()
        full_data = data_handler.fetch_data(SYMBOL, START_DATE, END_DATE)
        train_size = int(len(full_data) * 0.6)
        val_size = int(len(full_data) * 0.2)

        train_data = full_data.iloc[:train_size]
        val_data = full_data.iloc[train_size:train_size+val_size]
        test_data = full_data.iloc[train_size+val_size:]    
        
        print("\nInitializing training environment...")
        env = TradingEnvironment(train_data)
        
        # Initialize agent
        state_size = len(env.features)
        action_size = 3  # sell, hold, buy
        agent = DQNAgent(state_size, action_size)
        
        print("\nStarting training...")
        print(f"State size: {state_size}")
        print(f"Action size: {action_size}")
        
        # Train agent
        training_losses = train_agent(env, agent, EPISODES)
        
        print("\nStarting testing phase...")
        test_portfolio_values, test_actions = test_agent(agent, test_data, initial_balance=10000)
        
        # Plot results
        plot_results(env.portfolio_value_history, test_portfolio_values, training_losses)
        
        # Print final results
        print("\nTraining Results:")
        print(f"Initial Balance: ${100000:.2f}")
        print(f"Final Training Portfolio Value: ${env.portfolio_value_history[-1]:.2f}")
        print(f"Training Return: {((env.portfolio_value_history[-1] / 100000) - 1) * 100:.2f}%")
        
        print("\nTesting Results:")
        print(f"Initial Balance: ${10000:.2f}")
        print(f"Final Testing Portfolio Value: ${test_portfolio_values[-1]:.2f}")
        print(f"Testing Return: {((test_portfolio_values[-1] / 10000) - 1) * 100:.2f}%")
        
        # Save final model
        torch.save({
            'model_state_dict': agent.state_dict(),
            'optimizer_state_dict': agent.optimizer.state_dict(),
            'training_losses': training_losses,
            'train_portfolio_values': env.portfolio_value_history,
            'test_portfolio_values': test_portfolio_values,
        }, 'final_model.pth')
        
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise

# Improved Version👍

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device for computations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------------
# DataHandler and Aggregation
# -------------------------
class DataHandler:
    def __init__(self):
        self.scaler = MinMaxScaler()
    
    def fetch_data(self, symbol, start_date, end_date):
        print(f"Fetching data for {symbol} from {start_date} to {end_date}...")
        df = yf.download(symbol, start=start_date, end=end_date)
        if df.empty:
            raise ValueError(f"No data found for {symbol}")
        return self.prepare_data(df)
    
    def prepare_data(self, df):
        try:
            df = df.copy()
            # Preserve original close price for portfolio calculations
            df['CloseOrig'] = df['Close'].astype(float)
            
            # Convert price and volume arrays
            close_prices = df['Close'].values.astype(float).flatten()
            high_prices = df['High'].values.astype(float).flatten()
            low_prices = df['Low'].values.astype(float).flatten()
            volume = df['Volume'].values.astype(float).flatten()
            
            # Calculate technical indicators
            df['Returns'] = df['Close'].pct_change()
            df['SMA_20'] = talib.SMA(close_prices, timeperiod=20)
            df['EMA_20'] = talib.EMA(close_prices, timeperiod=20)
            df['RSI'] = talib.RSI(close_prices, timeperiod=14)
            
            macd, signal, _ = talib.MACD(close_prices)
            df['MACD'] = macd
            df['MACD_signal'] = signal
            
            bb_upper, bb_middle, bb_lower = talib.BBANDS(close_prices)
            df['BB_upper'] = bb_upper
            df['BB_middle'] = bb_middle
            df['BB_lower'] = bb_lower
            
            df['OBV'] = talib.OBV(close_prices, volume)
            df['MOM'] = talib.MOM(close_prices, timeperiod=14)
            
            # Features to be used as input; note: we do not normalize the original price
            features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                        'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 
                        'BB_lower', 'OBV', 'MOM']
            
            # Fill NA values
            df[features] = df[features].fillna(method='ffill').fillna(method='bfill')
            # Normalize only the features (not the CloseOrig)
            df[features] = self.scaler.fit_transform(df[features])
            
            print(f"Data shape after preparation: {df.shape}")
            return df.dropna()
        except Exception as e:
            print(f"Error in prepare_data: {str(e)}")
            raise

    def fetch_multiple_data(self, symbols, start_date, end_date):
        data_dict = {}
        for symbol in symbols:
            data_dict[symbol] = self.fetch_data(symbol, start_date, end_date)
        return data_dict

def aggregate_data(data_dict):
    """
    Aggregates the data across multiple symbols by taking the average of 
    features over the common date range.
    """
    common_index = None
    for df in data_dict.values():
        if common_index is None:
            common_index = df.index
        else:
            common_index = common_index.intersection(df.index)
    
    reindexed = []
    for symbol, df in data_dict.items():
        reindexed.append(df.loc[common_index])
    # Create a multi-index DataFrame (first level: symbol, second: feature)
    concatenated = pd.concat(reindexed, axis=1, keys=data_dict.keys())
    # Average over the symbols (grouping by the feature level)
    aggregated = concatenated.groupby(axis=1, level=1).mean()
    return aggregated

# -------------------------
# Trading Environment
# -------------------------
class TradingEnvironment:
    def __init__(self, data, initial_balance=100000):
        if data is None or data.empty:
            raise ValueError("Data cannot be empty")
        # Retain the original date index for testing visualization.
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI',
                         'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                         'BB_lower', 'OBV', 'MOM']
        if 'CloseOrig' not in data.columns:
            raise ValueError("Missing 'CloseOrig' column for price calculations")
        self.reset()
    
    def reset(self):
        self.balance = self.initial_balance
        self.position = 0.0  # Number of shares held
        self.current_step = 0
        self.portfolio_value_history = [self.initial_balance]
        return self._get_state()
    
    def _get_state(self):
        return self.data.iloc[self.current_step][self.features].values
    
    def step(self, action):
        # Current price comes from the raw close
        current_price = float(self.data.iloc[self.current_step]['CloseOrig'])
        prev_value = self.portfolio_value_history[-1]
        
        # Action space (5 discrete actions):
        # 0: Sell all position
        # 1: Sell half of position
        # 2: Hold (do nothing)
        # 3: Buy using 50% of available cash
        # 4: Buy using full available cash
        if current_price > 0:
            if action == 0 and self.position > 0:  # Sell all
                self.balance += current_price * self.position
                self.position = 0.0
            elif action == 1 and self.position > 0:  # Sell half
                shares_to_sell = self.position * 0.5
                self.balance += current_price * shares_to_sell
                self.position -= shares_to_sell
            elif action == 3 and self.balance > 0:  # Buy with 50% cash
                cash_to_use = self.balance * 0.5
                shares_to_buy = cash_to_use / current_price
                self.position += shares_to_buy
                self.balance -= current_price * shares_to_buy
            elif action == 4 and self.balance > 0:  # Buy with full cash
                shares_to_buy = self.balance / current_price
                self.position += shares_to_buy
                self.balance = 0
            # Action 2 is hold (do nothing)
        
        new_value = self.balance + (self.position * current_price)
        self.portfolio_value_history.append(new_value)
        reward = (new_value - prev_value) / prev_value if prev_value > 0 else 0
        
        self.current_step += 1
        done = (self.current_step >= len(self.data) - 1)
        next_state = self._get_state() if not done else np.zeros(len(self.features))
        return next_state, reward, done

# -------------------------
# DQN Agent
# -------------------------
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        
        # Neural network layers
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)
        
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()
        self.to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            act_values = self(state_tensor)
            return torch.argmax(act_values).item()
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return 0
        minibatch = random.sample(self.memory, self.batch_size)
        states = torch.FloatTensor([i[0] for i in minibatch]).to(device)
        actions = torch.LongTensor([i[1] for i in minibatch]).to(device)
        rewards = torch.FloatTensor([i[2] for i in minibatch]).to(device)
        next_states = torch.FloatTensor([i[3] for i in minibatch]).to(device)
        dones = torch.FloatTensor([i[4] for i in minibatch]).to(device)
        
        current_q = self(states).gather(1, actions.unsqueeze(1)).squeeze()
        with torch.no_grad():
            next_q = self(next_states).max(1)[0]
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        loss = self.criterion(current_q, target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
        return loss.item()

# -------------------------
# Training and Testing Functions
# -------------------------
def train_agent(env, agent, episodes):
    training_losses = []
    best_reward = float('-inf')
    
    for episode in tqdm(range(episodes), desc="Training"):
        state = env.reset()
        total_reward = 0
        episode_losses = []
        
        while True:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.replay()
            if loss > 0:
                episode_losses.append(loss)
            state = next_state
            total_reward += reward
            
            if done:
                avg_loss = np.mean(episode_losses) if episode_losses else 0
                training_losses.append(avg_loss)
                if total_reward > best_reward:
                    best_reward = total_reward
                    torch.save(agent.state_dict(), 'best_model.pth')
                print(f"\nEpisode: {episode+1}/{episodes} | Total Reward: {total_reward:.2f} | Avg Loss: {avg_loss:.4f} | Epsilon: {agent.epsilon:.4f} | Final Portfolio: ${env.portfolio_value_history[-1]:.2f}")
                break
    return training_losses

def test_agent(agent, test_data, initial_balance=10000):
    # Create a test environment subclassing TradingEnvironment
    class TestEnvironment(TradingEnvironment):
        def __init__(self, data, initial_balance):
            super().__init__(data, initial_balance)
    
    test_env = TestEnvironment(test_data, initial_balance)
    original_epsilon = agent.epsilon
    agent.epsilon = 0  # disable exploration for test
    state = test_env.reset()
    actions_taken = []
    
    done = False
    while not done:
        action = agent.act(state)
        actions_taken.append(action)
        state, reward, done = test_env.step(action)
    
    agent.epsilon = original_epsilon
    return test_env.portfolio_value_history, actions_taken

# -------------------------
# Plotting Functions
# -------------------------
def plot_results(train_values, test_values, training_losses):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(len(train_values))),
        y=train_values,
        mode='lines',
        name='Training Portfolio'
    ))
    fig.add_trace(go.Scatter(
        x=list(range(len(train_values), len(train_values)+len(test_values))),
        y=test_values,
        mode='lines',
        name='Testing Portfolio'
    ))
    fig.update_layout(title='Portfolio Value Over Time',
                      xaxis_title='Trading Steps',
                      yaxis_title='Portfolio Value ($)')
    
    loss_fig = go.Figure()
    loss_fig.add_trace(go.Scatter(
        x=list(range(len(training_losses))),
        y=training_losses,
        mode='lines',
        name='Training Loss'
    ))
    loss_fig.update_layout(title='Training Loss Over Episodes',
                           xaxis_title='Episode',
                           yaxis_title='Loss')
    
    fig.show()
    loss_fig.show()

def plot_test_signals(test_df, test_actions):
    # Use the original test data index and raw prices to overlay signals.
    dates = test_df.index
    prices = test_df['CloseOrig'].values
    buy_dates = []
    buy_prices = []
    sell_dates = []
    sell_prices = []
    
    # Map actions to signals:
    # 0 and 1 are Sell signals; 3 and 4 are Buy signals; 2 is Hold.
    for i, action in enumerate(test_actions):
        if action in [3, 4]:
            buy_dates.append(dates[i])
            buy_prices.append(prices[i])
        elif action in [0, 1]:
            sell_dates.append(dates[i])
            sell_prices.append(prices[i])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=prices, mode='lines', name='SP500 Index'))
    fig.add_trace(go.Scatter(x=buy_dates, y=buy_prices, mode='markers', name='Buy Signal',
                             marker=dict(color='green', size=10, symbol='triangle-up')))
    fig.add_trace(go.Scatter(x=sell_dates, y=sell_prices, mode='markers', name='Sell Signal',
                             marker=dict(color='red', size=10, symbol='triangle-down')))
    fig.update_layout(title='S&P500 Index Price with Trading Signals',
                      xaxis_title='Date',
                      yaxis_title='Price')
    fig.show()

# -------------------------
# Main Program
# -------------------------
if __name__ == "__main__":
    try:
        # Set seeds for reproducibility
        torch.manual_seed(42)
        np.random.seed(42)
        random.seed(42)
        
        # Define parameters
        TOP_10_SP500_STOCKS = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 'META', 'TSLA', 'BRK-B', 'UNH', 'XOM']
        INDEX_SYMBOL = '^GSPC'
        TRAIN_START_DATE = '2009-01-01'
        TRAIN_END_DATE = '2024-01-01'    # Approximately 15 years
        TEST_START_DATE = '2024-01-01'
        TEST_END_DATE = '2025-01-01'       # 1-year test period
        EPISODES = 100
        
        print("Initializing data handler and fetching training data for top-10 stocks...")
        data_handler = DataHandler()
        top10_data = data_handler.fetch_multiple_data(TOP_10_SP500_STOCKS, TRAIN_START_DATE, TRAIN_END_DATE)
        train_data = aggregate_data(top10_data)
        
        print("\nFetching testing data for the S&P500 index...")
        test_data = data_handler.fetch_data(INDEX_SYMBOL, TEST_START_DATE, TEST_END_DATE)
        
        print("\nInitializing training environment...")
        train_env = TradingEnvironment(train_data, initial_balance=100000)
        state_size = len(train_env.features)
        action_size = 5  # (0: Sell all, 1: Sell half, 2: Hold, 3: Buy with half cash, 4: Buy with full cash)
        print(f"State size: {state_size}, Action size: {action_size}")
        
        # Initialize agent
        agent = DQNAgent(state_size, action_size)
        
        print("\nStarting training...")
        training_losses = train_agent(train_env, agent, EPISODES)
        
        print("\nStarting testing phase on the S&P500 index...")
        test_portfolio_values, test_actions = test_agent(agent, test_data, initial_balance=10000)
        
        # Plot portfolio evolution and training loss
        plot_results(train_env.portfolio_value_history, test_portfolio_values, training_losses)
        
        # Visualize trading signals over the test period using raw index data
        plot_test_signals(test_data, test_actions)
        
        # Print final results
        print("\nTraining Results:")
        print(f"Initial Training Balance: $100000.00")
        print(f"Final Training Portfolio Value: ${train_env.portfolio_value_history[-1]:.2f}")
        training_return = ((train_env.portfolio_value_history[-1] / 100000) - 1) * 100
        print(f"Training Return: {training_return:.2f}%")
        
        print("\nTesting Results:")
        print(f"Initial Testing Balance: $10000.00")
        print(f"Final Testing Portfolio Value: ${test_portfolio_values[-1]:.2f}")
        testing_return = ((test_portfolio_values[-1] / 10000) - 1) * 100
        print(f"Testing Return: {testing_return:.2f}%")
        
        # Save final model and training statistics
        torch.save({
            'model_state_dict': agent.state_dict(),
            'optimizer_state_dict': agent.optimizer.state_dict(),
            'training_losses': training_losses,
            'train_portfolio_history': train_env.portfolio_value_history,
            'test_portfolio_history': test_portfolio_values,
            'test_actions': test_actions
        }, 'final_model.pth')
    
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise


Using device: cuda
Initializing data handler and fetching training data for top-10 stocks...
Fetching data for AAPL from 2009-01-01 to 2024-01-01...


[*********************100%***********************]  1 of 1 completed


Data shape after preparation: (3774, 17)
Fetching data for MSFT from 2009-01-01 to 2024-01-01...


[*********************100%***********************]  1 of 1 completed


Data shape after preparation: (3774, 17)
Fetching data for AMZN from 2009-01-01 to 2024-01-01...


...

Training:  99%|█████████▉| 99/100 [16:02<00:10, 10.70s/it]


Episode: 99/100 | Total Reward: 0.85 | Avg Loss: 0.0001 | Epsilon: 0.0100 | Final Portfolio: $213955.27


Training: 100%|██████████| 100/100 [16:12<00:00,  9.73s/it]


Episode: 100/100 | Total Reward: 0.94 | Avg Loss: 0.0000 | Epsilon: 0.0100 | Final Portfolio: $237083.59

Starting testing phase on the S&P500 index...



Training Results:
Initial Training Balance: $100000.00
Final Training Portfolio Value: $237083.59
Training Return: 137.08%

Testing Results:
Initial Testing Balance: $10000.00
Final Testing Portfolio Value: $11768.26
Testing Return: 17.68%


# Testing and Visualization of Improved Version over different indices

In [4]:
import numpy as np
import yfinance as yf
import talib
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Set the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------------
# DataHandler: Downloads and Prepares Test Data
# -------------------------
class DataHandler:
    def __init__(self):
        from sklearn.preprocessing import MinMaxScaler
        self.scaler = MinMaxScaler()

    def fetch_data(self, symbol, start_date, end_date):
        print(f"\nFetching data for {symbol} from {start_date} to {end_date}...")
        df = yf.download(symbol, start=start_date, end=end_date)
        if df.empty:
            raise ValueError(f"No data found for {symbol}")
        return self.prepare_data(df)

    def prepare_data(self, df):
        df = df.copy()
        # Preserve the original close prices for portfolio value calculation.
        df['CloseOrig'] = df['Close'].astype(float)
        
        # Convert necessary columns to one-dimensional float64 arrays to avoid dimension issues.
        close_prices = np.array(df['Close'], dtype=np.float64).flatten()
        volume = np.array(df['Volume'], dtype=np.float64).flatten()

        # Compute technical indicators.
        df['Returns'] = df['Close'].pct_change()
        df['SMA_20'] = talib.SMA(close_prices, timeperiod=20)
        df['EMA_20'] = talib.EMA(close_prices, timeperiod=20)
        df['RSI'] = talib.RSI(close_prices, timeperiod=14)
        macd, signal, _ = talib.MACD(close_prices)
        df['MACD'] = macd
        df['MACD_signal'] = signal
        bb_upper, bb_middle, bb_lower = talib.BBANDS(close_prices)
        df['BB_upper'] = bb_upper
        df['BB_middle'] = bb_middle
        df['BB_lower'] = bb_lower
        df['OBV'] = talib.OBV(close_prices, volume)
        df['MOM'] = talib.MOM(close_prices, timeperiod=14)

        # Features used for training (normalization applied to these)
        features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                    'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 
                    'BB_lower', 'OBV', 'MOM']
        df[features] = df[features].fillna(method='ffill').fillna(method='bfill')
        df[features] = self.scaler.fit_transform(df[features])
        df = df.dropna()
        print(f"Data shape after preparation: {df.shape}")
        return df

# -------------------------
# Trading Environment for Testing Mode
# -------------------------
class TradingEnvironment:
    def __init__(self, data, initial_balance=10000):
        if data.empty:
            raise ValueError("Data cannot be empty")
        # Reset index to simulate sequential trading.
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        # This list must match the feature names used in training.
        self.features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI',
                         'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                         'BB_lower', 'OBV', 'MOM']
        self.reset()

    def reset(self):
        self.balance = self.initial_balance
        self.position = 0.0  # Number (or proportion) of shares held.
        self.current_step = 0
        self.portfolio_value_history = [self.initial_balance]
        return self._get_state()

    def _get_state(self):
        return self.data.iloc[self.current_step][self.features].values

    def step(self, action):
        # Actual price for trading is taken from the unscaled close.
        current_price = float(self.data.iloc[self.current_step]['CloseOrig'])
        prev_value = self.portfolio_value_history[-1]

        # Define the 5-action space:
        # 0: Sell all, 1: Sell half, 2: Hold, 3: Buy with 50% of cash, 4: Buy with full cash.
        if current_price > 0:
            if action == 0 and self.position > 0:
                self.balance += current_price * self.position
                self.position = 0.0
            elif action == 1 and self.position > 0:
                shares_to_sell = self.position * 0.5
                self.balance += current_price * shares_to_sell
                self.position -= shares_to_sell
            elif action == 3 and self.balance > 0:
                cash_to_use = self.balance * 0.5
                shares_to_buy = cash_to_use / current_price
                self.position += shares_to_buy
                self.balance -= cash_to_use
            elif action == 4 and self.balance > 0:
                shares_to_buy = self.balance / current_price
                self.position += shares_to_buy
                self.balance = 0
            # Action 2 (Hold) does nothing.

        new_value = self.balance + (self.position * current_price)
        self.portfolio_value_history.append(new_value)
        reward = (new_value - prev_value) / prev_value if prev_value > 0 else 0

        self.current_step += 1
        done = (self.current_step >= len(self.data) - 1)
        next_state = self._get_state() if not done else np.zeros(len(self.features))
        return next_state, reward, done

# -------------------------
# DQNAgent: Loads Policy and Acts Greedily (epsilon = 0 during testing)
# -------------------------
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 0.0  # Deterministic policy during testing.
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32

        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()
        self.to(device)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

    def act(self, state):
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            action_values = self(state_tensor)
            return torch.argmax(action_values).item()

# -------------------------
# Test Simulation Function
# -------------------------
def test_agent(agent, env):
    state = env.reset()
    done = False
    actions_taken = []
    while not done:
        action = agent.act(state)
        actions_taken.append(action)
        state, reward, done = env.step(action)
    return env.portfolio_value_history, actions_taken

# -------------------------
# Plotting Function
# -------------------------
def plot_results(portfolio_values, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(len(portfolio_values))),
        y=portfolio_values,
        mode='lines+markers',
        name='Portfolio Value'
    ))
    fig.update_layout(title=title,
                      xaxis_title='Trading Days',
                      yaxis_title='Portfolio Value ($)')
    fig.show()

# -------------------------
# Main Routine: Load Model and Test on Multiple Indexes
# -------------------------
if __name__ == '__main__':
    try:
        # List of indexes to test; include at least five.
        index_list = ['^DJI', '^GSPC', '^FTSE', '^NSEI', '^N225']
        TEST_START_DATE = '2024-01-01'
        TEST_END_DATE = '2025-01-01'
        TEST_INITIAL_BALANCE = 10000

        # Load the pre-trained model once.
        state_size = 12    # Must match training state vector dimension.
        action_size = 5    # 5-action space used during training.
        agent = DQNAgent(state_size, action_size)
        model_path = 'final_model.pth'
        checkpoint = torch.load(model_path, map_location=device)
        agent.load_state_dict(checkpoint['model_state_dict'])
        agent.eval()
        print("Model loaded successfully.")

        # Initialize a DataHandler (common for all tests).
        data_handler = DataHandler()

        # Test the model on each index.
        for symbol in index_list:
            try:
                test_data = data_handler.fetch_data(symbol, TEST_START_DATE, TEST_END_DATE)
                test_env = TradingEnvironment(test_data, initial_balance=TEST_INITIAL_BALANCE)
                portfolio_history, actions_taken = test_agent(agent, test_env)
                final_value = portfolio_history[-1]
                testing_return = ((final_value/TEST_INITIAL_BALANCE) - 1) * 100
                title = f"{symbol} Portfolio Value (Return: {testing_return:.2f}%)"
                plot_results(portfolio_history, title)
                print(f"\nTesting Results for {symbol}:")
                print(f"  Initial Balance: ${TEST_INITIAL_BALANCE:.2f}")
                print(f"  Final Portfolio Value: ${final_value:.2f}")
                print(f"  Return: {testing_return:.2f}%")
            except Exception as inner_e:
                print(f"Error testing on {symbol}: {inner_e}")

    except Exception as e:
        print(f"Error during testing: {e}")
        raise


Using device: cuda


[*********************100%***********************]  1 of 1 completed

Model loaded successfully.

Fetching data for ^DJI from 2024-01-01 to 2025-01-01...
Data shape after preparation: (252, 17)


[*********************100%***********************]  1 of 1 completed


Testing Results for ^DJI:
  Initial Balance: $10000.00
  Final Portfolio Value: $10709.70
  Return: 7.10%

Fetching data for ^GSPC from 2024-01-01 to 2025-01-01...
Data shape after preparation: (252, 17)


...

[*********************100%***********************]  1 of 1 completed


Testing Results for ^FTSE:
  Initial Balance: $10000.00
  Final Portfolio Value: $10166.23
  Return: 1.66%

Fetching data for ^NSEI from 2024-01-01 to 2025-01-01...
Data shape after preparation: (246, 17)


[*********************100%***********************]  1 of 1 completed


Testing Results for ^NSEI:
  Initial Balance: $10000.00
  Final Portfolio Value: $10799.99
  Return: 8.00%

Fetching data for ^N225 from 2024-01-01 to 2025-01-01...
Data shape after preparation: (245, 17)



Testing Results for ^N225:
  Initial Balance: $10000.00
  Final Portfolio Value: $12851.42
  Return: 28.51%


# Training over Multiple Indices for Better Performance🎭

In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ---------------------------
# DataHandler Class
# ---------------------------
class DataHandler:
    def __init__(self):
        from sklearn.preprocessing import MinMaxScaler
        self.scaler = MinMaxScaler()
    
    def fetch_data(self, symbol, start_date, end_date):
        print(f"Fetching data for {symbol} from {start_date} to {end_date}...")
        df = yf.download(symbol, start=start_date, end=end_date)
        if df.empty:
            raise ValueError(f"No data found for {symbol}")
        return self.prepare_data(df)
    
    def prepare_data(self, df):
        df = df.copy()
        # Preserve the original closing price for portfolio simulation
        df['CloseOrig'] = df['Close'].astype(float)
        
        # Convert columns explicitly to one-dimensional arrays (to avoid TA-Lib errors)
        close_prices = np.array(df['Close'], dtype=np.float64).flatten()
        volume = np.array(df['Volume'], dtype=np.float64).flatten()
        
        # Compute technical indicators
        df['Returns'] = df['Close'].pct_change()
        df['SMA_20'] = talib.SMA(close_prices, timeperiod=20)
        df['EMA_20'] = talib.EMA(close_prices, timeperiod=20)
        df['RSI'] = talib.RSI(close_prices, timeperiod=14)
        macd, signal, _ = talib.MACD(close_prices)
        df['MACD'] = macd
        df['MACD_signal'] = signal
        bb_upper, bb_middle, bb_lower = talib.BBANDS(close_prices)
        df['BB_upper'] = bb_upper
        df['BB_middle'] = bb_middle
        df['BB_lower'] = bb_lower
        df['OBV'] = talib.OBV(close_prices, volume)
        df['MOM'] = talib.MOM(close_prices, timeperiod=14)
        
        # List of features to normalize (do not scale CloseOrig)
        features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                    'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 
                    'BB_lower', 'OBV', 'MOM']
        df[features] = df[features].fillna(method='ffill').fillna(method='bfill')
        df[features] = self.scaler.fit_transform(df[features])
        df = df.dropna()
        print(f"Prepared data shape for {df.index[-1]} rows and {len(df.columns)} columns.")
        return df
    
    def fetch_multiple_data(self, symbols, start_date, end_date):
        data_dict = {}
        for symbol in symbols:
            try:
                data = self.fetch_data(symbol, start_date, end_date)
                data_dict[symbol] = data
            except Exception as e:
                print(f"Error fetching {symbol}: {e}")
        return data_dict

# ---------------------------
# Aggregate Data Function
# ---------------------------
def aggregate_data(data_dict):
    """
    Given a dictionary of DataFrames (keyed by symbol), find the common date
    index and average the features.
    """
    common_index = None
    for df in data_dict.values():
        if common_index is None:
            common_index = df.index
        else:
            common_index = common_index.intersection(df.index)
    reindexed = []
    for symbol, df in data_dict.items():
        reindexed.append(df.loc[common_index])
    concatenated = pd.concat(reindexed, axis=1, keys=data_dict.keys())
    # Average features across symbols; this applies to every column including CloseOrig.
    aggregated = concatenated.groupby(axis=1, level=1).mean()
    return aggregated

# ---------------------------
# Trading Environment Class
# ---------------------------
class TradingEnvironment:
    def __init__(self, data, initial_balance=100000):
        if data is None or data.empty:
            raise ValueError("Data cannot be empty")
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        # Features used as state (these are the normalized technical indicators)
        self.features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI',
                         'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                         'BB_lower', 'OBV', 'MOM']
        # "CloseOrig" is used for actual portfolio value calculation.
        if 'CloseOrig' not in self.data.columns:
            raise ValueError("Missing 'CloseOrig' column in data.")
        self.reset()
    
    def reset(self):
        self.balance = self.initial_balance
        self.position = 0.0  # shares held
        self.current_step = 0
        self.portfolio_value_history = [self.initial_balance]
        return self._get_state()
    
    def _get_state(self):
        return self.data.iloc[self.current_step][self.features].values
    
    def step(self, action):
        # Retrieve actual price from the unscaled "CloseOrig"
        current_price = float(self.data.iloc[self.current_step]['CloseOrig'])
        prev_value = self.portfolio_value_history[-1]
        # Action space (5 discrete actions):
        # 0: Sell all, 1: Sell half, 2: Hold, 3: Buy with 50% cash, 4: Buy with full cash.
        if current_price > 0:
            if action == 0 and self.position > 0:
                # Sell all
                self.balance += current_price * self.position
                self.position = 0.0
            elif action == 1 and self.position > 0:
                # Sell half
                shares_to_sell = self.position * 0.5
                self.balance += current_price * shares_to_sell
                self.position -= shares_to_sell
            elif action == 3 and self.balance > 0:
                # Buy with 50% cash
                cash_to_use = self.balance * 0.5
                shares_to_buy = cash_to_use / current_price
                self.position += shares_to_buy
                self.balance -= cash_to_use
            elif action == 4 and self.balance > 0:
                # Buy with full cash
                shares_to_buy = self.balance / current_price
                self.position += shares_to_buy
                self.balance = 0
            # Action 2 (Hold) does nothing.
        new_value = self.balance + (self.position * current_price)
        self.portfolio_value_history.append(new_value)
        reward = (new_value - prev_value) / prev_value if prev_value > 0 else 0
        self.current_step += 1
        done = (self.current_step >= len(self.data) - 1)
        next_state = self._get_state() if not done else np.zeros(len(self.features))
        return next_state, reward, done

# ---------------------------
# DQNAgent Class
# ---------------------------
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0  # initial exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32

        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()
        self.to(device)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            action_values = self(state_tensor)
            return torch.argmax(action_values).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return 0
        minibatch = random.sample(self.memory, self.batch_size)
        states = torch.FloatTensor([m[0] for m in minibatch]).to(device)
        actions = torch.LongTensor([m[1] for m in minibatch]).to(device)
        rewards = torch.FloatTensor([m[2] for m in minibatch]).to(device)
        next_states = torch.FloatTensor([m[3] for m in minibatch]).to(device)
        dones = torch.FloatTensor([m[4] for m in minibatch]).to(device)
        current_q = self(states).gather(1, actions.unsqueeze(1)).squeeze()
        with torch.no_grad():
            next_q = self(next_states).max(1)[0]
            target_q = rewards + (1 - dones) * self.gamma * next_q
        loss = self.criterion(current_q, target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss.item()

# ---------------------------
# Training Function
# ---------------------------
def train_agent(agent, training_datasets, episodes):
    training_losses = []
    best_reward = float('-inf')
    
    # Training proceeds across episodes.
    # In each episode we randomly choose one aggregated dataset (i.e. one country) 
    # and simulate a complete trading episode over its available historical period.
    for episode in tqdm(range(episodes), desc="Training Episodes"):
        # Randomly choose one country's aggregated data
        country, data = random.choice(list(training_datasets.items()))
        env = TradingEnvironment(data, initial_balance=100000)
        state = env.reset()
        total_reward = 0
        episode_losses = []
        while True:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.replay()
            if loss:
                episode_losses.append(loss)
            state = next_state
            total_reward += reward
            if done:
                avg_loss = np.mean(episode_losses) if episode_losses else 0
                training_losses.append(avg_loss)
                if total_reward > best_reward:
                    best_reward = total_reward
                    torch.save(agent.state_dict(), 'final_model.pth')
                print(f"\nEpisode {episode+1}/{episodes} | Country: {country} | Total Reward: {total_reward:.2f} | Avg Loss: {avg_loss:.4f} | Epsilon: {agent.epsilon:.4f} | Final Portfolio: ${env.portfolio_value_history[-1]:.2f}")
                break
    return training_losses

# ---------------------------
# Main Routine for Training on Multiple Indexes
# ---------------------------
if __name__ == '__main__':
    try:
        # Define training period with a larger dataset
        TRAIN_START_DATE = '1995-01-01'
        TRAIN_END_DATE   = '2024-01-01'
        EPISODES = 100  # adjust number of episodes as needed
        
        # Dictionary of indexes and their top 10 companies (tickers)
        index_companies = {
            "USA": ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 'META', 'TSLA', 'BRK-B', 'UNH', 'XOM'],
            "India": ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'LT.NS', 'AXISBANK.NS', 'ITC.NS', 'HINDUNILVR.NS'],
            "Japan": ['7203.T', '6758.T', '9984.T', '8306.T', '6902.T', '9432.T', '7267.T', '7974.T', '6501.T', '8801.T'],
            "UK": ['HSBA.L', 'BP.L', 'VOD.L', 'GSK.L', 'RIO.L', 'BT-A.L', 'ULVR.L', 'DGE.L', 'AZN.L', 'BATS.L'],
            "France": ['OR.PA', 'MC.PA', 'SAN.PA', 'AI.PA', 'BNP.PA', 'DG.PA', 'EN.PA', 'RI.PA', 'KER.PA', 'SU.PA']
        }
        
        # ... rest of your training code remains unchanged ...
        
        data_handler = DataHandler()
        training_datasets = {}
        
        # For each country/index, fetch data for its top companies and aggregate.
        for country, tickers in index_companies.items():
            print(f"\nProcessing {country} data:")
            company_data = data_handler.fetch_multiple_data(tickers, TRAIN_START_DATE, TRAIN_END_DATE)
            if company_data:
                agg_data = aggregate_data(company_data)
                training_datasets[country] = agg_data
            else:
                print(f"No valid data fetched for {country}.")
        
        if not training_datasets:
            raise ValueError("No aggregated training data available from any index!")
        
        # All training datasets must have the expected technical indicator columns.
        # We assume each aggregated DataFrame contains at least the following columns:
        # ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 'MACD', 'MACD_signal', 'BB_upper', 'BB_middle', 'BB_lower', 'OBV', 'MOM', 'CloseOrig']
        
        # Initialize the training environment parameters are set within the episode loop.
        state_size = 12  # corresponds to the feature vector (excluding CloseOrig)
        action_size = 5  # five possible actions
        
        # Initialize the DQN agent
        agent = DQNAgent(state_size, action_size)
        
        print("\nStarting training on multiple indexes...")
        training_losses = train_agent(agent, training_datasets, EPISODES)
        
        print("\nTraining Completed. Best model saved as final_model.pth")
    
    except Exception as e:
        print(f"Error during training: {e}")
        raise


[*********************100%***********************]  1 of 1 completed

Using device: cuda

Processing USA data:
Fetching data for AAPL from 1995-01-01 to 2024-01-01...
Prepared data shape for 2023-12-29 00:00:00 rows and 17 columns.
Fetching data for MSFT from 1995-01-01 to 2024-01-01...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Prepared data shape for 2023-12-29 00:00:00 rows and 17 columns.
Fetching data for AMZN from 1995-01-01 to 2024-01-01...
Prepared data shape for 2023-12-29 00:00:00 rows and 17 columns.
Fetching data for NVDA from 1995-01-01 to 2024-01-01...
Prepared data shape for 2023-12-29 00:00:00 rows and 17 columns.
Fetching data for GOOGL from 1995-01-01 to 2024-01-01...
Prepared data shape for 2023-12-29 00:00:00 rows and 17 columns.
Fetching data for META from 1995-01-01 to 2024-01-01...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...


Episode 98/100 | Country: Japan | Total Reward: 0.98 | Avg Loss: 0.0001 | Epsilon: 0.0100 | Final Portfolio: $215342.60


Training Episodes:  99%|█████████▉| 99/100 [23:03<00:13, 13.67s/it]


Episode 99/100 | Country: UK | Total Reward: 3.56 | Avg Loss: 0.0063 | Epsilon: 0.0100 | Final Portfolio: $13554.52


Training Episodes: 100%|██████████| 100/100 [23:10<00:00, 13.90s/it]


Episode 100/100 | Country: USA | Total Reward: 0.92 | Avg Loss: 0.0000 | Epsilon: 0.0100 | Final Portfolio: $236717.10

Training Completed. Best model saved as final_model.pth


# Testing the Saved Model (Returns and Porfolio Visualization)

In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Set the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------------
# DataHandler: Downloads and Prepares Test Data
# -------------------------
class DataHandler:
    def __init__(self):
        from sklearn.preprocessing import MinMaxScaler
        self.scaler = MinMaxScaler()
        
    def fetch_data(self, symbol, start_date, end_date):
        print(f"\nFetching data for {symbol} from {start_date} to {end_date}...")
        df = yf.download(symbol, start=start_date, end=end_date)
        if df.empty:
            raise ValueError(f"No data found for {symbol}")
        return self.prepare_data(df)
    
    def prepare_data(self, df):
        df = df.copy()
        # Preserve original closing price for portfolio calculations
        df['CloseOrig'] = df['Close'].astype(float)
        
        # Convert to one-dimensional float64 numpy arrays
        close_prices = np.array(df['Close'], dtype=np.float64).flatten()
        volume = np.array(df['Volume'], dtype=np.float64).flatten()
        
        # Compute technical indicators
        df['Returns'] = df['Close'].pct_change()
        df['SMA_20'] = talib.SMA(close_prices, timeperiod=20)
        df['EMA_20'] = talib.EMA(close_prices, timeperiod=20)
        df['RSI'] = talib.RSI(close_prices, timeperiod=14)
        macd, signal, _ = talib.MACD(close_prices)
        df['MACD'] = macd
        df['MACD_signal'] = signal
        bb_upper, bb_middle, bb_lower = talib.BBANDS(close_prices)
        df['BB_upper'] = bb_upper
        df['BB_middle'] = bb_middle
        df['BB_lower'] = bb_lower
        df['OBV'] = talib.OBV(close_prices, volume)
        df['MOM'] = talib.MOM(close_prices, timeperiod=14)
        
        # Define state representation features (do not scale CloseOrig)
        features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                    'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                    'BB_lower', 'OBV', 'MOM']
        # Fill missing values and normalize
        df[features] = df[features].fillna(method='ffill').fillna(method='bfill')
        df[features] = self.scaler.fit_transform(df[features])
        df = df.dropna()
        print(f"Data shape after preparation: {df.shape}")
        return df

# -------------------------
# Trading Environment for Testing
# -------------------------
class TradingEnvironment:
    def __init__(self, data, initial_balance=10000):
        if data.empty:
            raise ValueError("Data cannot be empty")
        # Reset index for sequential simulation
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        # Must match training state features
        self.features = ['Close', 'Returns', 'SMA_20', 'EMA_20', 'RSI', 
                         'MACD', 'MACD_signal', 'BB_upper', 'BB_middle',
                         'BB_lower', 'OBV', 'MOM']
        if 'CloseOrig' not in self.data.columns:
            raise ValueError("Missing 'CloseOrig' column.")
        self.reset()
    
    def reset(self):
        self.balance = self.initial_balance
        self.position = 0.0  # Number of shares held
        self.current_step = 0
        self.portfolio_value_history = [self.initial_balance]
        return self._get_state()
    
    def _get_state(self):
        return self.data.iloc[self.current_step][self.features].values
    
    def step(self, action):
        # Retrieve the unscaled current price for trading decisions.
        current_price = float(self.data.iloc[self.current_step]['CloseOrig'])
        prev_value = self.portfolio_value_history[-1]
        
        # Action mapping (5 discrete actions):
        # 0: Sell all, 1: Sell half, 2: Hold
        # 3: Buy with 50% available cash, 4: Buy with full available cash.
        if current_price > 0:
            if action == 0 and self.position > 0:
                self.balance += current_price * self.position
                self.position = 0.0
            elif action == 1 and self.position > 0:
                shares_to_sell = self.position * 0.5
                self.balance += current_price * shares_to_sell
                self.position -= shares_to_sell
            elif action == 3 and self.balance > 0:
                cash_to_use = self.balance * 0.5
                shares_to_buy = cash_to_use / current_price
                self.position += shares_to_buy
                self.balance -= cash_to_use
            elif action == 4 and self.balance > 0:
                shares_to_buy = self.balance / current_price
                self.position += shares_to_buy
                self.balance = 0
                
        new_value = self.balance + (self.position * current_price)
        self.portfolio_value_history.append(new_value)
        
        reward = (new_value - prev_value) / prev_value if prev_value > 0 else 0
        
        self.current_step += 1
        done = (self.current_step >= len(self.data) - 1)
        next_state = self._get_state() if not done else np.zeros(len(self.features))
        return next_state, reward, done

# -------------------------
# DQNAgent: Defines the Network & Action Selection (Greedy for Testing)
# -------------------------
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        # Set epsilon=0 for testing (greedy policy)
        self.epsilon = 0.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_size)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()
        self.to(device)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
    def act(self, state):
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            action_values = self(state_tensor)
            return torch.argmax(action_values).item()

# -------------------------
# Test Simulation Function: Returns Portfolio History, Actions, and Signals
# -------------------------
def test_agent(agent, env):
    state = env.reset()
    done = False
    actions_taken = []
    signals = []  # record "buy", "sell", or "hold" for each step.
    while not done:
        action = agent.act(state)
        actions_taken.append(action)
        # Define buy signals for actions 3 & 4, sell signals for actions 0 & 1.
        if action in [3, 4]:
            signals.append("buy")
        elif action in [0, 1]:
            signals.append("sell")
        else:
            signals.append("hold")
        state, reward, done = env.step(action)
    return env.portfolio_value_history, actions_taken, signals

# -------------------------
# Plotting Functions
# -------------------------
def plot_portfolio(portfolio_values, signals, title):
    # Prepare markers for buy and sell signals along the portfolio evolution
    buy_indices = [i for i, s in enumerate(signals) if s == "buy"]
    sell_indices = [i for i, s in enumerate(signals) if s == "sell"]
    buy_values = [portfolio_values[i] for i in buy_indices]
    sell_values = [portfolio_values[i] for i in sell_indices]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(len(portfolio_values))),
        y=portfolio_values,
        mode='lines+markers',
        name='Portfolio Value'
    ))
    fig.add_trace(go.Scatter(
        x=buy_indices,
        y=buy_values,
        mode='markers',
        name='Buy Signal',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))
    fig.add_trace(go.Scatter(
        x=sell_indices,
        y=sell_values,
        mode='markers',
        name='Sell Signal',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))
    fig.update_layout(title=title,
                      xaxis_title='Trading Steps',
                      yaxis_title='Portfolio Value ($)')
    fig.show()

def plot_price_signals(test_data, signals, title):
    # Plot price (CloseOrig) with buy and sell markers.
    # Use the environment's reset data (i.e. a sequential index)
    prices = test_data['CloseOrig'].values
    steps = list(range(len(prices)))
    
    buy_indices = [i for i, s in enumerate(signals) if s == "buy"]
    sell_indices = [i for i, s in enumerate(signals) if s == "sell"]
    buy_prices = [prices[i] for i in buy_indices]
    sell_prices = [prices[i] for i in sell_indices]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=steps,
        y=prices,
        mode='lines',
        name='Price'
    ))
    fig.add_trace(go.Scatter(
        x=buy_indices,
        y=buy_prices,
        mode='markers',
        name='Buy Signal',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))
    fig.add_trace(go.Scatter(
        x=sell_indices,
        y=sell_prices,
        mode='markers',
        name='Sell Signal',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))
    fig.update_layout(title=title,
                      xaxis_title='Trading Steps',
                      yaxis_title='Price ($)')
    fig.show()

# -------------------------
# Main Testing Routine
# -------------------------
if __name__ == '__main__':
    try:
        # Dictionary of indexes to test: name -> ticker symbol
        indexes = {
            "USA_S&P500": "^GSPC",        # S&P 500 Index
            "India_Nifty50": "^NSEI",      # Nifty 50 Index
            "Japan_Nikkei225": "^N225",    # Nikkei 225 Index
            "UK_FTSE100": "^FTSE",         # FTSE 100 Index
            "France_CAC40": "^FCHI"        # CAC 40 Index
        }
        # Testing period of one year:
        TEST_START_DATE = '2024-01-01'
        TEST_END_DATE   = '2025-01-01'
        TEST_INITIAL_BALANCE = 10000
        
        # Load the pre-trained model.
        state_size = 12  # Must match state dimensions used in training.
        action_size = 5  # Five discrete actions.
        agent = DQNAgent(state_size, action_size)
        model_path = 'final_model.pth'
        checkpoint = torch.load(model_path, map_location=device)
        agent.load_state_dict(checkpoint)  # Adjust if checkpoint is nested.
        agent.eval()
        print("Pre-trained model loaded successfully.")
        
        # Initialize the DataHandler.
        data_handler = DataHandler()
        
        # Test the model on each index.
        for index_name, ticker in indexes.items():
            try:
                print("\n============================================")
                print(f"Testing on {index_name} ({ticker})")
                test_data = data_handler.fetch_data(ticker, TEST_START_DATE, TEST_END_DATE)
                # Initialize the TradingEnvironment using the prepared data.
                env = TradingEnvironment(test_data, initial_balance=TEST_INITIAL_BALANCE)
                portfolio_history, actions_taken, signals = test_agent(agent, env)
                final_value = portfolio_history[-1]
                return_pct = ((final_value / TEST_INITIAL_BALANCE) - 1) * 100
                print(f"Initial Balance: ${TEST_INITIAL_BALANCE:.2f}")
                print(f"Final Portfolio Value: ${final_value:.2f}")
                print(f"Cumulative Return: {return_pct:.2f}%")
                
                title_portfolio = f"{index_name} Portfolio Evolution (Return: {return_pct:.2f}%)"
                plot_portfolio(portfolio_history, signals, title_portfolio)
                
                title_price = f"{index_name} Price with Buy/Sell Signals"
                # Use the original test_data (not reset) for price plotting with the same sequential order.
                test_data_reset = test_data.reset_index(drop=True)
                plot_price_signals(test_data_reset, signals, title_price)
            except Exception as inner_ex:
                print(f"Error testing on {ticker}: {inner_ex}")
        
    except Exception as e:
        print(f"Error during testing: {e}")
        raise


[*********************100%***********************]  1 of 1 completed

Using device: cuda
Pre-trained model loaded successfully.

Testing on USA_S&P500 (^GSPC)

Fetching data for ^GSPC from 2024-01-01 to 2025-01-01...
Data shape after preparation: (252, 17)
Initial Balance: $10000.00
Final Portfolio Value: $12916.94
Cumulative Return: 29.17%


[*********************100%***********************]  1 of 1 completed


Testing on India_Nifty50 (^NSEI)

Fetching data for ^NSEI from 2024-01-01 to 2025-01-01...
Data shape after preparation: (246, 17)
Initial Balance: $10000.00
Final Portfolio Value: $10871.45
Cumulative Return: 8.71%


[*********************100%***********************]  1 of 1 completed


Testing on Japan_Nikkei225 (^N225)

Fetching data for ^N225 from 2024-01-01 to 2025-01-01...
Data shape after preparation: (245, 17)
Initial Balance: $10000.00
Final Portfolio Value: $13740.45
Cumulative Return: 37.40%


[*********************100%***********************]  1 of 1 completed


Testing on UK_FTSE100 (^FTSE)

Fetching data for ^FTSE from 2024-01-01 to 2025-01-01...
Data shape after preparation: (254, 17)
Initial Balance: $10000.00
Final Portfolio Value: $10602.08
Cumulative Return: 6.02%


[*********************100%***********************]  1 of 1 completed


Testing on France_CAC40 (^FCHI)

Fetching data for ^FCHI from 2024-01-01 to 2025-01-01...
Data shape after preparation: (256, 17)
Initial Balance: $10000.00
Final Portfolio Value: $10109.86
Cumulative Return: 1.10%
